### import needed packages

In [1]:
import requests as r
from bs4 import BeautifulSoup
import urllib.parse
import pandas as pd
import pprint



###  making a list of column name
which doesn't need to be iterated every time <br>
list name : column_names

In [2]:
# making a fixed column names list

column_name_unformat = []
column_names = []

column_names.append('date') 

# getting the url
page_url = "http://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=C0A9G0&stname=%25E5%258D%2597%25E6%25B8%25AF&datepicker=2016-02-01"
res = r.get(page_url)
res.encoding = 'utf-8' # 調整Response obect的encoding，不然中文是亂碼
soup = BeautifulSoup(res.text, 'html5lib')

for i in range(9, 24):
    col_title_unformat = soup.select("th")[i]
    col_title = "%s"%col_title_unformat
    
    column_name_unformat.append(col_title)
    #col_title = col_title.split("<th>")[1]
    #print(col_title)


for i in column_name_unformat :
    col_title = i.split('<th>')[1].split("</th>")[0].replace("<br/>", " ")
    
    column_names.append(col_title)
    
    #print(col_title)

   
column_names

['date',
 '觀測時間 (LST) ObsTime',
 '測站氣壓 (hPa) StnPres',
 '海平面氣壓 (hPa) SeaPres',
 '氣溫 (℃) Temperature',
 '露點溫度 (℃) Td dew point',
 '相對溼度 (%) RH',
 '風速 (m/s) WS',
 '風向 (360degree) WD',
 '最大陣風 (m/s) WSGust',
 '最大陣風風向 (360degree) WDGust',
 '降水量 (mm) Precp',
 '降水時數 (hr) PrecpHour ',
 '日照時數 (hr) SunShine',
 '全天空日射量 (MJ/㎡) GloblRad',
 '能見度 (km) Visb']

### making a crawler that can crawl over date pages (for a whole month)

In [ ]:
page_url = "http://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=C0A9G0&stname=%25E5%258D%2597%25E6%25B8%25AF&datepicker={}"
date = []

month=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
year = [2015, 2016]

for y in year :
    
    

    for m in range(1,len(month)+1) :
        
        tr_list = []
        
        if m<10 :
            m = "0"+"%s"%m
        else :
            m = "%s"%m
        #print(m)
    
        if y == year[0] :
            if m in ['01', '03', '05', '07', '08', '10', '12'] :
                day = 31
            elif m == '02' :
                day = 28
            elif m in ['04', '06', '09', '11'] :
                day = 30
            
            
        elif y == year[1] :
            if m in ['01', '03', '05', '07', '08', '10', '12'] :
                day = 31
            elif m == '02' :
                day = 29
            elif m in ['04', '06', '09', '11'] :
                day = 30
            
            
        
        
    
        
    
        for d in range(1, day+1) :
            #print('======={}===={}============='.format(y, m))
            if d<10 :
                d = "0"+"%s"%d
        
            #print(d)
        
            
            ymd_date = "{}-{}-{}".format(y, m, d)
            #print(ymd_date)
        
            
            each_page = page_url.format(ymd_date)
            #print(each_page)
            
            each_res = r.get(each_page)
            each_res.encoding = "utf-8"
            soup = BeautifulSoup(each_res.text, 'html5lib')
            
            # getting out the data
            # every hour in one day
            try :
                for i in range(3, 27):
                                                       
                    ii =soup.select("tr" )[i]
                    tr_set = "%s"%ii
                    tr_set = tr_set.replace("\n\t\t", "").replace("\xa0", "").replace("<tr>", "")\
                    .replace("</tr>", "").replace('<td nowrap="">', "").replace("</td>", ",").replace("<td>", "")\
                    .replace("\t", "").replace("\n\t", '').replace('\n', '').replace(" ", "")
                    # tr_set is still a string now
                    tr_set = tr_set.split(",") 
                    # for this time, after the action of splitting, tr_set becomes a list !
                    
                    tr_set.insert(0,ymd_date)
                    tr_set.pop(16)
                    tr_list.append(tr_set)
            except :
                print("no record for %s"%ymd_date)
                continue
                
        #tr_list
        df = pd.DataFrame(tr_list , columns = column_names)
        
        #print(df)
        m = int(m)
        #print(m)
        df.to_csv('./data/{}_{}{}_weather_data.csv'.format(y, m, month[m-1]))
